In [ ]:
Collect and extract features from google trends

In [1]:
# import wikipedia
# print(wikipedia.summary("Wikipedia"))



from datetime import datetime
import time,sys
from pathlib import Path
import time
import pickle
from urllib.request import Request, urlopen
import pandas as pd
from pytrends.request import TrendReq
from datetime import date, datetime, timedelta
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
#function to fet trends geographical data
#due to inability to comapre countried and technologies on the same query due to google limitation
#this function is currently unused
def get_googletrends_stats(name):
    pytrend.build_payload(kw_list=[name],timeframe="2016-08-28 2021-08-15")
    # Interest by Region
    df = pytrend.interest_by_region()
    trends_dict = {"trends_region___" + str(key): val for key, val in df.to_dict()[name].items()}
    pytrend.build_payload(kw_list=[name])
    trends_dict["trends_region___High_level_countries"]=pytrend.interest_by_region(inc_low_vol=False, inc_geo_code=False).sum().values[0]
    
    
    return trends_dict
    
    

# Load tech file

In [3]:
tech_df=pd.read_csv('https://docs.google.com/spreadsheets/d/1BZ_NP8PJbDsIXz4cbjSHlILQ8yBfy0vXcqm5Zw4fNoU/export?format=csv',
                   # Set first column as rownames in data frame
                   index_col=0,
                  )
tech_list=tech_df.loc[tech_df['Emerging Tech'].notnull(),'Emerging Tech'].to_list()
#tech_list

# Google trends scrap

In [4]:
#set pytrend agent
pytrend = TrendReq()

In [ ]:
#test single query to check if we can still scrap
start_date = date(2000, 1, 1)
end_date=date(2021, 10, 1)

start = time.time()
start_date_str = start_date.strftime('%Y-%m-%d')
# dict_key=f"{key}_{i}_{start_date_str}"
date_iterator=end_date
date_iterator_str = date_iterator.strftime('%Y-%m-%d')
tf = start_date_str + ' ' + date_iterator_str
pytrend.build_payload(kw_list=tech_list[0:5],cat=0,timeframe=tf)

dff=pytrend.interest_over_time()
display(dff)

In [ ]:
#set dict to collect terms data
#load from temp pickle file if continue from the middle of a previous run
trend_cat_dict={}
# with open('trends_cat.pickle', 'rb') as handle:
#     trend_cat_dict = pickle.load(handle)

In [ ]:
#collect trends data for each terms

#basic categories to collect
cat_ids={"All":0,"Science":174,"Law & Government":19,"News":16,"Finance": 7,"Business & Industrial": 12}

#names to ignore
exculde_list=[]
delta=5 #size of term group, query 5 terms at a time
for key, value in cat_ids.items():
    if key not in exculde_list:
        for i in range(0,len(tech_list),delta):
            start=i
            end=i+delta
            if end>len(tech_list):
                end=len(tech_list)
            l=tech_list[start:end]
            l = list(dict.fromkeys(l))
            start_date = date(2004, 1, 1)
            end_date=date(2021, 10, 1)           
            dict_key=f"{key}_{i}"
            
            if dict_key not in trend_cat_dict:
                print(dict_key,start_date_str,date_iterator_str,l)
                start_date_str = start_date.strftime('%Y-%m-%d')
                date_iterator_str = end_date.strftime('%Y-%m-%d')
                tf = start_date_str + ' ' + date_iterator_str
                pytrend.build_payload(kw_list=l,cat=value,timeframe=tf)
                dff=pytrend.interest_over_time()
                try:
                    dff=dff.drop(['isPartial',], axis=1)
                except:
                    print("no partial")
                trend_cat_dict[dict_key]=dff.T
                time.sleep(1)

                

In [ ]:
#save to pickle
with open('trends_cat.pickle', 'wb') as handle:
    pickle.dump(trend_cat_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#save raw trends temporal data
data_path=f"features/trend_categories/"
#         Path(data_path).mkdir(parents=True, exist_ok=True)
for cat_key, cat_value in cat_ids.items():
    print(cat_key)
    filtered_dict = {k:v for k,v in trend_cat_dict.items() if cat_key in k}
    dft=pd.concat(filtered_dict.values())
    dft.fillna(0).to_excel(f"{data_path}{cat_key}_{datetime.now().strftime('%d%m%Y%H%M%S')}.xlsx") 
    display(dft.head())
   


In [ ]:
trend_cat_dict.keys()

Ignore rest of file, contain tests for regional trends data

# test# 

In [ ]:
start_date = date(2011, 1, 1)
end_date=date(2021, 1, 1)



print(start_date,i,len(tech_list))
date_iterator=start_date+ relativedelta(months=1)-timedelta(days=1)
start_date_str = start_date.strftime('%Y-%m-%d')
date_iterator_str = date_iterator.strftime('%Y-%m-%d')
tf = start_date_str + ' ' + date_iterator_str
pytrend.build_payload(kw_list=l,cat=value,timeframe=tf)
dff=pytrend.interest_by_region(resolution='COUNTRY', inc_low_vol=True, inc_geo_code=False)
dff

# Google trends region scrap

In [5]:
trend_loc_df={}
with open('trends_loc.pickle', 'rb') as handle:
    trend_loc_df = pickle.load(handle)

In [8]:
start_mes = time.time()
delta=5




for i in range(215,len(tech_list),delta):
    start=i
    end=i+delta
    if end>len(tech_list):
        end=len(tech_list)
    l=tech_list[start:end]
    l = list(dict.fromkeys(l))
    start_date = date(2004, 1, 1)
    end_date=date(2021, 10, 1)           
    
    
    
    while start_date<end_date:
        start_date_str = start_date.strftime('%Y-%m-%d')
        dict_key=f"{i}_{start_date_str}"
        
        if dict_key not in trend_loc_df:
            print(dict_key,start_date_str)
            date_iterator=start_date+relativedelta(months=1)-timedelta(days=1)
            date_iterator_str = date_iterator.strftime('%Y-%m-%d')
            tf = start_date_str + ' ' + date_iterator_str
            pytrend.build_payload(kw_list=l,timeframe=tf)
            dff=pytrend.interest_by_region()
#             try:
#                 dff=dff.drop(['isPartial',], axis=1)
#             except:
#                 print("no partial")
            trend_loc_df[dict_key]=dff.T
            time.sleep(1)
        start_date=start_date+relativedelta(months=1)
                
                
# for i in range(0,len(tech_list),delta):
#     while start_date<end_date:
#         temp_date=
#         delta=5
#         print(start_date,i)

#         start=i
#         end=i+delta
#         if end>len(tech_list):
#             end=len(tech_list)
#         l=tech_list[start:end]
#         pytrend.build_payload(kw_list=l,timeframe=f"{start_date.strftime('%Y-%m-%d')} {temp_date.strftime('%Y-%m-%d')}")
#         trend_loc_df[start_date.strftime('%Y-%m-%d')]
#         time.sleep(60)
        
#     start_date=start_date+timedelta(months=1)

# end = time.time()
# print(end - start_mes)
# data_path=f"features/trend_categories/"
# Path(data_path).mkdir(parents=True, exist_ok=True)
# trend_loc_df_concat=pd.concat(trend_loc_df, axis=1)
# trend_loc_df_concat=trend_loc_df_concat.drop(['isPartial'], axis=1).T
# trend_loc_df_concat.to_excel(f"{data_path}{key}_{datetime.now().strftime('%d%m%Y%H%M%S')}.xlsx")
# trend_loc_df_concat

215_2004-01-01 2004-01-01
215_2004-02-01 2004-02-01
215_2004-03-01 2004-03-01
215_2004-04-01 2004-04-01
215_2004-05-01 2004-05-01
215_2004-06-01 2004-06-01
215_2004-07-01 2004-07-01
215_2004-08-01 2004-08-01
215_2004-09-01 2004-09-01
215_2004-10-01 2004-10-01
215_2004-11-01 2004-11-01
215_2004-12-01 2004-12-01
215_2005-01-01 2005-01-01
215_2005-02-01 2005-02-01
215_2005-03-01 2005-03-01
215_2005-04-01 2005-04-01
215_2005-05-01 2005-05-01
215_2005-06-01 2005-06-01
215_2005-07-01 2005-07-01
215_2005-08-01 2005-08-01
215_2005-09-01 2005-09-01
215_2005-10-01 2005-10-01
215_2005-11-01 2005-11-01
215_2005-12-01 2005-12-01
215_2006-01-01 2006-01-01
215_2006-02-01 2006-02-01
215_2006-03-01 2006-03-01
215_2006-04-01 2006-04-01
215_2006-05-01 2006-05-01
215_2006-06-01 2006-06-01
215_2006-07-01 2006-07-01
215_2006-08-01 2006-08-01
215_2006-09-01 2006-09-01
215_2006-10-01 2006-10-01
215_2006-11-01 2006-11-01
215_2006-12-01 2006-12-01
215_2007-01-01 2007-01-01
215_2007-02-01 2007-02-01
215_2007-03-

ReadTimeout: HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)

In [ ]:
with open('trends_loc.pickle', 'wb') as handle:
    pickle.dump(trend_loc_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
len(tech_list)

In [ ]:
pytrend = TrendReq()
tech_list=tech_df.loc[tech_df['Emeging Tech'].notnull(),'Emeging Tech'].to_list()
trend_cat_df=[]

In [ ]:
trends_region_dict={}

In [ ]:
scrap=True
trends_dict
if scrap:
    count=0
    start=0
    for index, row in tech_df.iloc[start:,:].iterrows():              
        ##trends farm
        if trend_farm:
            name = row['Emeging Tech']
            if isinstance(name, str):
                print(index,name)
                trends_region_dict[name]=get_googletrends_stats(name)
                

In [ ]:
with open('trends_region_dict.pickle', 'wb') as handle:
        pickle.dump(trends_region_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
trends_region_dict={}
with open('trends_region_dict.pickle', 'rb') as handle:
    trends_region_dict = pickle.load(handle)

final_tech_df=pd.DataFrame.from_dict(trends_region_dict).T
#final_tech_df=final_tech_df.loc[final_tech_df['wiki___ID'].notnull(),:]
final_tech_df.to_excel('trends_region_dict.xlsx')

feature_df=final_tech_df.index.to_frame()
final_tech_df

In [ ]:
eu_list=['Ukraine',
'France',
'Spain',
'Sweden',
'Germany',
'Finland',
'Norway',
'Poland',
'Italy',
'United Kingdom',
'Romania',
'Belarus',
'Greece',
'Bulgaria',
'Iceland',
'Portugal',
'Denmark',
'Hungary',
'Serbia',
'Austria',
'Ireland',
'Lithuania',
'Latvia',
'Croatia',
'Bosnia & Herzegovina',
'Slovakia',
'Estonia',
'Netherlands',
'Switzerland',
'Moldova',
'Belgium',
'Albania',
'North Macedonia',
'Slovenia',
'Montenegro',
'Cyprus',
'Luxembourg',
'Faroe Islands',
'Andorra',
'Malta',
'Liechtenstein',
'Guernsey',
'San Marino',
'Gibraltar',
'Monaco',
'Vatican City',
'United Kingdom'
]
eu_list=["trends_region___" + sub for sub in eu_list]



In [ ]:
feature_df['trends_N_of_countries(total)']=final_tech_df.filter(regex='^trends_region',axis=1).sum(axis=1)


feature_df['trends_N_of_countries(only high volume)']=final_tech_df['trends_region___High_level_countries']

feature_df['trends_N _of_country (EU+UK)']=final_tech_df[eu_list].sum(axis=1)

feature_df['trends_N _of_country(Russia)']=final_tech_df['trends_region___Russia']

feature_df['trends_N _of_country(China)']=final_tech_df['trends_region___China']

feature_df['trends_N _of_country(US)']=final_tech_df['trends_region___United States']

feature_df['trends_N_of_country(Japan+Souht Korea+Singapur+Indonesia)']=final_tech_df['trends_region___Japan']\
+final_tech_df['trends_region___Indonesia']\
+final_tech_df['trends_region___South Korea']\
+final_tech_df['trends_region___Singapore']

feature_df['trends_N _of_country(Israel)']=final_tech_df['trends_region___Israel']

feature_df['trends_N _of_country(UAE)']=final_tech_df['trends_region___United Arab Emirates']

feature_df['trends_N _of_country(Canada+Australia+New_Zeland)']=final_tech_df['trends_region___Canada']\
+final_tech_df['trends_region___Australia']\
+final_tech_df['trends_region___New Zealand']

feature_df['trends_N _of_country(India)']=final_tech_df['trends_region___India']

In [ ]:
for name in final_tech_df.columns:
    data_path=f"features/trend_regions/{name}/"
    Path(data_path).mkdir(parents=True, exist_ok=True)
    print(name,data_path)
    final_tech_df[name].to_frame().to_excel(f"{data_path}{name}_{datetime.now().strftime('%d%m%Y%H%M%S')}.xlsx")

In [ ]:
for name in feature_df.columns:
    data_path=f"features/trend_regions/{name}/"
    Path(data_path).mkdir(parents=True, exist_ok=True)
    print(name,data_path)
    feature_df[name].to_frame().to_excel(f"{data_path}{name}_{datetime.now().strftime('%d%m%Y%H%M%S')}.xlsx")

# Wikipedia scrap

In [ ]:
tech_dict={}

In [ ]:
scrap=True

if scrap:
    count=0
    start=384
    for index, row in tech_df.iloc[start:,:].iterrows():
        exclude_names_list = ['Page size', 'Total edits']
        # filtered_dict = get_wiki_stats('carbon_nanotube', [])
        # for key in filtered_dict.keys():
        #     print(key,":",filtered_dict[key])



        wiki_tech_df=tech_df[tech_df['wiki name'].notnull()]
        wiki_dict = {}
        wikimedia_dict = {}
        trends_dict={}
        
        
        ##wikifarm
        if wiki_farm:
            name=row['wiki name']
            print(index,name)
            if isinstance(name,str):
                wiki_dict=get_wiki_stats(name, exclude_names_list)
                wiki_dict = {"wiki___" + str(key): val for key, val in wiki_dict.items()}
                
        ##wikimediafarm
        if wikimedia_farm:
            name=row['wiki name']
            if isinstance(name,str):
                wikimedia_dict=get_wikimedia_stats(name,)
                wikimedia_dict = {"wikimedia___" + str(key): val for key, val in wikimedia_dict.items()}
                time.sleep(5)
                count+=1
                if count==10:
                    count=0
                    time.sleep(300)
                    
                
#         ##trends farm
#         if trend_farm:
#             name = row['Emeging Tech']
#             if isinstance(name, str):
#                 print(name)
#                 trends_dict=get_googletrends_stats(name)
                
#         if index>6:
#             break

        wiki_dict.update(wikimedia_dict)
#         wiki_dict.update(trends_dict)
        tech_dict[name]=wiki_dict



        

    with open('wiki.pickle', 'wb') as handle:
        pickle.dump(tech_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
tech_dict={}
with open('wiki.pickle', 'rb') as handle:
    tech_dict = pickle.load(handle)

final_tech_df=pd.DataFrame.from_dict(tech_dict).T
final_tech_df=final_tech_df.loc[final_tech_df['wiki___ID'].notnull(),:]
final_tech_df.to_excel('wiki.xlsx')

feature_df=final_tech_df.index.to_frame()
final_tech_df

Wikipedia

In [ ]:
feature_df['wiki_N_of_total_edits']=final_tech_df['wiki___Minor edits'].str.split("·",0).str[0].astype('int64')
feature_df['wiki_N_of_total_edits']+=final_tech_df['wiki___IP edits'].str.split("·",0).str[0].astype('int64')
feature_df['wiki_N_of_total_edits']+=final_tech_df['wiki___Bot edits'].str.split("·",0).str[0].astype('int64')
feature_df['wiki_N_of_total_edits']+=final_tech_df['wiki___(Semi-)automated edits'].astype('int64')
feature_df['wiki_N_of_total_edits']+=final_tech_df['wiki___Reverted edits'].astype('int64')

feature_df['wiki_N_of_editors']=final_tech_df['wiki___Editors'].astype('int64')

feature_df['wiki_N_of_files']=final_tech_df['wiki___Files'].astype('int64')

feature_df['wiki_N_of_categories']=final_tech_df['wiki___Categories'].astype('int64')

feature_df['wiki_N_of_external_links']=final_tech_df['wiki___External links'].astype('int64')

feature_df['wiki_N_of_income_links']=final_tech_df['wiki___Links to this page'].astype('int64')

feature_df['wiki_N_of_redirects']=final_tech_df['wiki___Redirects'].astype('int64')

feature_df['wiki_N_of_outcome_links']=final_tech_df['wiki___Links from this page'].astype('int64')

feature_df['wiki_N_of_pageviewes']=final_tech_df['wiki___Pageviews (60 days)'].astype('int64')

feature_df['wiki_N_of_averege time btween_edits']=final_tech_df['wiki___Average time between edits (days)'].str.split(" ",0).str[0]

feature_df['wiki_N_of_averege _edits_per_day']=final_tech_df['wiki___Average edits per day'].astype('float')

feature_df['wiki_N_of_averege _edits_made by top 10% editors']=final_tech_df['wiki___Edits made by the top 10% of editors'].str.split("·",0).str[0]

feature_df['wiki_N_of_Characters']=final_tech_df['wiki___Characters'].astype('int64')

feature_df['wiki_N_of_sections']=final_tech_df['wiki___Sections'].astype('int64')

feature_df['wiki_N_of_references']=final_tech_df['wiki___References'].astype('int64')

feature_df['wiki_N_of_Unique references']=final_tech_df['wiki___Unique references'].astype('int64')

feature_df['wiki_N_of_bot_edits']=final_tech_df['wiki___Bot edits'].str.split("·",0).str[0].astype('int64')

feature_df['wiki_N_of_a_major_edits']=final_tech_df['wiki___Minor edits'].str.split("·",0).str[0].astype('int64')-feature_df['wiki_N_of_total_edits'].astype('int64')

feature_df['wiki_N_of_pagewatchers']=final_tech_df['wiki___Page watchers']

feature_df['wiki_N_of_languages']=len(final_tech_df["wikimedia___langlinks"])

In [ ]:
for name in feature_df.columns:
    data_path=f"features/wiki/{name}/"
    Path(data_path).mkdir(parents=True, exist_ok=True)
    print(name,data_path)
    feature_df[name].to_frame().to_excel(f"{data_path}{name}_{datetime.now().strftime('%d%m%Y%H%M%S')}.xlsx")

In [ ]:
for name in final_tech_df.columns:
    data_path=f"features/wiki/{name}/"
    Path(data_path).mkdir(parents=True, exist_ok=True)
    print(name,data_path)
    final_tech_df[name].to_frame().to_excel(f"{data_path}{name}_{datetime.now().strftime('%d%m%Y%H%M%S')}.xlsx")

In [ ]:
final_tech_df

In [ ]:
feature_df['wiki_N_of_rel_WikiProjects']=final_tech_df['wiki___Editors'].astype('int64')

In [ ]:
feature_df['wiki_N_of_rewuse']=final_tech_df['wiki___Editors'].astype('int64')

In [ ]:
feature_df['trends_%out_of_total_serches']=final_tech_df['trends_region___India']
feature_df['trends_%out_of_total_serches_by category(science)']=final_tech_df['trends_region___India']
feature_df['trends_%out_of_total_serches_by category(news)']=final_tech_df['trends_region___India']
feature_df['trends_%out_of_total_serches_by category(Law_and_Order)']=final_tech_df['trends_region___India']
feature_df['trends_%out_of_total_serches_by category(finance)']=final_tech_df['trends_region___India']
feature_df['trends_%out_of_total_serches_by category(Bussines_and_ Industry)']=final_tech_df['trends_region___India']


